In [1]:
from models.MLP_dir import MLP_dir
from models.MLP_incr import MLP_incr
from models.LSTM_dir import LSTM_dir
from models.tmp import EP_NN
from models.commons import MLP, LSTM
from data.materials import load_responses
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import os.path as osp

def data_to_tensor(x:np.array):
    return torch.tensor(
        x.astype(np.float32),
        dtype=torch.float32
    ).unsqueeze(-1)

In [2]:
def train_model(model: MLP_dir | MLP_incr | LSTM_dir, mat_name, inp_name): 
    eps_list, sig_list = load_responses(mat_name,'random',inp_name,data_path='data')

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    y_train, y_tmp, u_train, u_tmp = train_test_split(y_list, u_list, test_size=0.3, random_state=42)
    y_val, y_test, u_val, u_test = train_test_split(y_tmp, u_tmp, test_size=0.5, random_state=42)

    model.fit(
        epochs=500,
        y_train=y_train, u_train=u_train,
        y_val=y_val,u_val=u_val,
        early_stopping=True,
        patience=20,
        min_delta=1e-6,
        verbose=True,
        shuffle=False,
    )

    return model.evaluate(y_test,u_test).item()

In [16]:
model = MLP_dir(2,8,2,seed=42)

test_err = train_model(
    model,
    mat_name = 'isotropic-swift',
    inp_name = 'pd_ms_42_200',
)

print(f"{test_err*100} %")

Epoch 1/500, Loss: 0.19004340842366219, Val Loss: 0.09363675278921922, LR: 0.001
Epoch 2/500, Loss: 0.06166707544720599, Val Loss: 0.04151262783755859, LR: 0.001
Epoch 3/500, Loss: 0.03284184353958283, Val Loss: 0.02447239194686214, LR: 0.001
Epoch 4/500, Loss: 0.0215030981959509, Val Loss: 0.016970889487614233, LR: 0.001
Epoch 5/500, Loss: 0.015258547929780824, Val Loss: 0.012266002936909597, LR: 0.001
Epoch 6/500, Loss: 0.011124707377582256, Val Loss: 0.009101314741807679, LR: 0.001
Epoch 7/500, Loss: 0.008388663557291563, Val Loss: 0.007018596058090528, LR: 0.001
Epoch 8/500, Loss: 0.006610632859103914, Val Loss: 0.005657534580677748, LR: 0.001
Epoch 9/500, Loss: 0.005436733102292887, Val Loss: 0.004721632456251731, LR: 0.001
Epoch 10/500, Loss: 0.004602862144487777, Val Loss: 0.004032258045238753, LR: 0.001
Epoch 11/500, Loss: 0.0039740015411682955, Val Loss: 0.0035047210364912945, LR: 0.001
Epoch 12/500, Loss: 0.0034793854383419133, Val Loss: 0.0030862729259145756, LR: 0.001
Epoch

In [17]:
def eval_model(model: MLP_dir | MLP_incr | LSTM_dir, mat_name, inp_type, inp_name, save_plot=False):
    
    eps_list, sig_list = load_responses(
        mat_name,inp_type,inp_name,
        data_path='data'
    )

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    return model.evaluate(
        y_list,u_list,
        save_plot=save_plot,
        path=osp.join(f'tmp/{inp_name}.png') # TODO: finish path
    ).item()

In [18]:
mat_name = 'isotropic-swift'

eval_list = []
for inp_name in ['amplitude', 'cyclic', 'impulse', 'piecewise']:
    eval_list += [
        eval_model(model,mat_name,'static',inp_name,save_plot=True)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

for inp_name in ['bl_ms_42_200', 'gp_42_200', 'pd_ms_42_200', 'rw_42_200']:
    eval_list += [
        eval_model(model,mat_name,'random',inp_name,save_plot=False)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

print(f'\nMean: {np.mean(eval_list)*100:02f} %')

amplitude: 3.940310 %
cyclic: 5.845309 %
impulse: 3.702134 %
piecewise: 4.931023 %
bl_ms_42_200: 1.773494 %
gp_42_200: 2.948822 %
pd_ms_42_200: 1.032426 %
rw_42_200: 4.804168 %

Mean: 3.622211 %


(0, 200, False): 3.656732 %

(1e-4, 200, False): 3.622211 %

(1e-4, 200, True): 3.656732 %

(1e-2, 200, False): 4.618025 %

(1e-4, 1000, False): 64.602013 %

(1e-2, 1000, False): 6.192588 %

7.349098 % 7-8-3 MLP_incr, 56 seed

3.622211 % 2-8-2 MLP_dir, 42 seed

In [2]:
model = EP_NN(2,8,2, incr=False, network=MLP, seed=42)

In [3]:
mat_name = 'isotropic-swift'
inp_name = 'pd_ms_42_200'

eps_list, sig_list = load_responses(mat_name,'random',inp_name,data_path='data')

y_list = data_to_tensor(sig_list)
u_list = data_to_tensor(eps_list)

y_train, y_tmp, u_train, u_tmp = train_test_split(y_list, u_list, test_size=0.3, random_state=42)
y_val, y_test, u_val, u_test = train_test_split(y_tmp, u_tmp, test_size=0.5, random_state=42)

model.fit(
    epochs=500,
    y_train=y_train, u_train=u_train,
    y_val=y_val,u_val=u_val,
    batch_size=1,
    early_stopping=True,
    patience=20,
    min_delta=1e-6,
    verbose=True,
    shuffle=False,
)

Epoch 1/500, Loss: 0.19004340842366219, Val Loss: 0.09363675278921922, LR: 0.001
Epoch 2/500, Loss: 0.06166707544720599, Val Loss: 0.04151262783755859, LR: 0.001
Epoch 3/500, Loss: 0.03284184353958283, Val Loss: 0.02447239194686214, LR: 0.001
Epoch 4/500, Loss: 0.0215030981959509, Val Loss: 0.016970889487614233, LR: 0.001
Epoch 5/500, Loss: 0.015258547929780824, Val Loss: 0.012266002936909597, LR: 0.001
Epoch 6/500, Loss: 0.011124707377582256, Val Loss: 0.009101314741807679, LR: 0.001
Epoch 7/500, Loss: 0.008388663557291563, Val Loss: 0.007018596058090528, LR: 0.001
Epoch 8/500, Loss: 0.006610632859103914, Val Loss: 0.005657534580677748, LR: 0.001
Epoch 9/500, Loss: 0.005436733102292887, Val Loss: 0.004721632456251731, LR: 0.001
Epoch 10/500, Loss: 0.004602862144487777, Val Loss: 0.004032258045238753, LR: 0.001
Epoch 11/500, Loss: 0.0039740015411682955, Val Loss: 0.0035047210364912945, LR: 0.001
Epoch 12/500, Loss: 0.0034793854383419133, Val Loss: 0.0030862729259145756, LR: 0.001
Epoch

In [4]:
def eval_model(model:EP_NN, mat_name, inp_type, inp_name, save_plot=False):
    
    eps_list, sig_list = load_responses(
        mat_name,inp_type,inp_name,
        data_path='data'
    )

    y_list = data_to_tensor(sig_list)
    u_list = data_to_tensor(eps_list)

    return model.evaluate(
        y_list,u_list,
        save_plot=save_plot,
        path=osp.join(f'tmp/{inp_name}.png') # TODO: finish path
    ).item()

In [5]:
mat_name = 'isotropic-swift'

eval_list = []
for inp_name in ['amplitude', 'cyclic', 'impulse', 'piecewise']:
    eval_list += [
        eval_model(model,mat_name,'static',inp_name,save_plot=True)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

for inp_name in ['bl_ms_42_200', 'gp_42_200', 'pd_ms_42_200', 'rw_42_200']:
    eval_list += [
        eval_model(model,mat_name,'random',inp_name,save_plot=False)
    ]
    print(f'{inp_name}: {eval_list[-1]*100:02f} %')

print(f'\nMean: {np.mean(eval_list)*100:02f} %')

amplitude: 3.940310 %
cyclic: 5.845309 %
impulse: 3.702134 %
piecewise: 4.931023 %
bl_ms_42_200: 1.773494 %
gp_42_200: 2.948822 %
pd_ms_42_200: 1.032426 %
rw_42_200: 4.804168 %

Mean: 3.622211 %
